In [1]:
import sqlalchemy as db

In [2]:
engine = db.create_engine('sqlite:///mydb.db')

In [3]:
conn = engine.connect()

In [4]:
metadata = db.MetaData()

In [18]:
cars = db.Table(
    'Cars', metadata,
    db.Column('car_id', db.Integer, primary_key=True),
    db.Column('car_name', db.Text,),
    db.Column('car_number', db.Text),
    db.Column('car_color', db.Text),
)


In [44]:
cars

Table('Cars', MetaData(), Column('car_id', Integer(), table=<Cars>, primary_key=True, nullable=False), Column('car_name', Text(), table=<Cars>), Column('car_number', Text(), table=<Cars>), Column('car_color', Text(), table=<Cars>), schema=None)

In [6]:
drivers = db.Table(
    'Drivers', metadata,
    db.Column('driver_id', db.Integer, primary_key = True),
    db.Column('driver_name', db.Text),
    db.Column('driver_phonenumber', db.Text),
    db.Column('driver_rating', db.Integer),
    db.Column('car_id', db.Integer, db.ForeignKey('Cars.car_id'))
)

In [7]:
users = db.Table(
    'Users', metadata,
    db.Column('user_id', db.Integer, primary_key = True),
    db.Column('user_name', db.Text),
    db.Column('user_phonenumber', db.Text),
    db.Column('user_rating', db.Integer)
)


In [8]:
orders = db.Table(
    'Orders', metadata,
    db.Column('order_id', db.Integer, primary_key = True),
    db.Column('user_id', db.Integer, db.ForeignKey('Users.user_id')),
    db.Column('driver_id', db.Integer, db.ForeignKey('Drivers.driver_id')),
    db.Column('cost', db.Numeric(precision=3, scale=2))
)

In [46]:
metadata.create_all(engine)

In [45]:
cars_insert = cars.insert().values([
    {'car_name': 'Volkswagen', 'car_number': '9416', 'car_color': 'blue'},
    {'car_name': 'Toyota', 'car_number': '3246', 'car_color': 'black'},
    {'car_name': 'Aston Martin', 'car_number': '4568', 'car_color': 'green'},
    {'car_name': 'Volkswagen', 'car_number': '7951', 'car_color': 'white'},
    {'car_name': 'Audi', 'car_number': '9152', 'car_color': 'grey'},
    {'car_name': 'Audi', 'car_number': '4562', 'car_color': 'black'}

])

In [47]:
conn.execute(cars_insert)

In [11]:
drivers_insert = drivers.insert().values([
    {'driver_name': 'Jack', 'driver_phonenumber': '2146513', 'driver_rating': 5, 'car_id': 1},
    {'driver_name': 'Sam', 'driver_phonenumber': '7845354', 'driver_rating': 5, 'car_id': 5},
    {'driver_name': 'Din', 'driver_phonenumber': '1354878', 'driver_rating': 5, 'car_id': 3},
    {'driver_name': 'Castiel', 'driver_phonenumber': '3220511', 'driver_rating': 5, 'car_id': 2},
    {'driver_name': 'Crowley', 'driver_phonenumber': '666', 'driver_rating': 4, 'car_id': 4},
    {'driver_name': 'Lucifer', 'driver_phonenumber': '66666666', 'driver_rating': 3, 'car_id': 6}
])

In [21]:
conn.execute(drivers_insert)

In [12]:
users_insert = users.insert().values([
    {'user_name': 'Rowena', 'user_phonenumber': '5456512','user_rating': 5},
    {'user_name': 'Bobby', 'user_phonenumber': '8795154','user_rating': 5},
    {'user_name': 'Kevin', 'user_phonenumber': '2653164','user_rating': 5},
    {'user_name': 'John', 'user_phonenumber': '1584269','user_rating': 3},
    {'user_name': 'Mary', 'user_phonenumber': '1257863','user_rating': 4}

])

In [22]:
conn.execute(users_insert)

In [13]:
orders_insert = orders.insert().values([
    {'user_id': 1, 'driver_id': 5, 'cost': 25.15},
    {'user_id': 5, 'driver_id': 4, 'cost': 52.11},
    {'user_id': 1, 'driver_id': 1, 'cost': 2.14},
    {'user_id': 1, 'driver_id': 3, 'cost': 4.42},
    {'user_id': 2, 'driver_id': 5, 'cost': 12.80},
    {'user_id': 2, 'driver_id': 2, 'cost': 13.40},
    {'user_id': 2, 'driver_id': 4, 'cost': 19.90},
    {'user_id': 4, 'driver_id': 1, 'cost': 28.15},
    {'user_id': 3, 'driver_id': 2, 'cost': 3.12},
    {'user_id': 3, 'driver_id': 2, 'cost': 9.70},
    {'user_id': 5, 'driver_id': 3, 'cost': 6.66},
    {'user_id': 2, 'driver_id': 1, 'cost': 8.42},
])

In [23]:
conn.execute(orders_insert)

In [59]:
select_all = db.select(users)
select_result = conn.execute(select_all)
select_result.fetchall()

[(1, 'Rowena', '5456512', 5),
 (2, 'Bobby', '8795154', 5),
 (3, 'Kevin', '2653164', 5),
 (4, 'John', '1584269', 3),
 (5, 'Mary', '1257863', 4)]

select из таблицы drivers для водителей с рейтингом 5

In [53]:
select1 = db.select(drivers).where(drivers.columns.driver_rating == 5)
select_result = conn.execute(select1)
select_result.fetchall()


[(1, 'Jack', '2146513', 5, 1),
 (2, 'Sam', '7845354', 5, 5),
 (3, 'Din', '1354878', 5, 3),
 (4, 'Castiel', '3220511', 5, 2),
 (7, 'Jody', '2141542', 5, 7),
 (8, 'Donna', '5544473', 5, 9)]

 **использование having**
выбрать пользователей, сумма заказов которых превышает 20

In [39]:
select2 = db.select(orders.columns.user_id, db.func.sum(orders.columns.cost)).group_by(orders.columns.user_id).having(db.func.sum(orders.columns.cost)>20)
select_result = conn.execute(select2)
select_result.fetchall()


[(1, Decimal('31.71')),
 (2, Decimal('54.52')),
 (4, Decimal('28.15')),
 (5, Decimal('58.77'))]

**использование left join**

In [60]:
select3 = db.select(drivers.join(cars, isouter = True))
select_result = conn.execute(select3)
select_result.fetchall()

[(1, 'Jack', '2146513', 5, 1, 1, 'Volkswagen', '9416', 'blue'),
 (2, 'Sam', '7845354', 5, 5, 5, 'Audi', '9152', 'grey'),
 (3, 'Din', '1354878', 5, 3, 3, 'Aston Martin', '4568', 'green'),
 (4, 'Castiel', '3220511', 5, 2, 2, 'Toyota', '3246', 'black'),
 (5, 'Crowley', '666', 4, 4, 4, 'Volkswagen', '7951', 'white'),
 (6, 'Lucifer', '66666666', 3, 6, 6, 'Audi', '4562', 'black'),
 (7, 'Jody', '2141542', 5, 7, None, None, None, None),
 (8, 'Donna', '5544473', 5, 9, None, None, None, None)]

**использование inner join**

In [63]:
select4 = db.select(drivers.join(cars))
select_result = conn.execute(select4)
select_result.fetchall()

[(1, 'Jack', '2146513', 5, 1, 1, 'Volkswagen', '9416', 'blue'),
 (2, 'Sam', '7845354', 5, 5, 5, 'Audi', '9152', 'grey'),
 (3, 'Din', '1354878', 5, 3, 3, 'Aston Martin', '4568', 'green'),
 (4, 'Castiel', '3220511', 5, 2, 2, 'Toyota', '3246', 'black'),
 (5, 'Crowley', '666', 4, 4, 4, 'Volkswagen', '7951', 'white'),
 (6, 'Lucifer', '66666666', 3, 6, 6, 'Audi', '4562', 'black')]

**Использование union**

In [72]:
union_select = db.union(db.select(users.columns.user_name, users.columns.user_rating).where(users.columns.user_rating == 5),
                        db.select(drivers.columns.driver_name, drivers.columns.driver_rating).where(drivers.columns.driver_rating == 4))
select_result = conn.execute(union_select)
select_result.fetchall()

[('Bobby', 5), ('Crowley', 4), ('Kevin', 5), ('Rowena', 5)]

**Использование подзапросов**: объединение таблицы пользователей с таблицей, где посчитали общую стоимость заказов и выбрали те, которые превышают 20 (см. выше)

In [73]:
select5 = db.select(users.join(db.select(orders.columns.user_id, db.func.sum(orders.columns.cost)).group_by(orders.columns.user_id).having(db.func.sum(orders.columns.cost)>20), isouter = True))
select_result = conn.execute(select5)
select_result.fetchall()

<ipython-input-73-937e6c8355bd>:1: SADeprecationWarning: Implicit coercion of SELECT and textual SELECT constructs into FROM clauses is deprecated; please call .subquery() on any Core select or ORM Query object in order to produce a subquery object.
  select5 = db.select(users.join(db.select(orders.columns.user_id, db.func.sum(orders.columns.cost)).group_by(orders.columns.user_id).having(db.func.sum(orders.columns.cost)>20), isouter = True))


[(1, 'Rowena', '5456512', 5, 1, Decimal('31.71')),
 (2, 'Bobby', '8795154', 5, 2, Decimal('54.52')),
 (3, 'Kevin', '2653164', 5, None, None),
 (4, 'John', '1584269', 3, 4, Decimal('28.15')),
 (5, 'Mary', '1257863', 4, 5, Decimal('58.77'))]